# Function for pulling articles from News API and Gnews API using different keywords




### Defining modules, endpoints, and keys

In [9]:
#loading modules
import pandas as pd
import itertools as tl
import csv
import requests
import json
import pprint
import numpy as np

#defining two different apis
apis = ['gnews', 'news']

#defining endpoints
gnews_url = "https://gnews.io/api/v3/search"
news_url = "https://newsapi.org/v2/everything?"


#defining keys
gnews_key = 'c16c4a07e49fceb90fcd30d1752f7040' #

news_key = "0c86ea54fc744a6eb7a45501d912e00e" #Praveen's key

### Defining inner functions

In [10]:
#making a function to create different combinations
def combinations(keyword_dict):
    t = []
    for y in keyword_dict.values():
        t.append(y)
    if keyword_dict == obj:
        gnews_combo = []
        for words in tl.product(*t):
            gnews_combo.append(words)
        return gnews_combo
    elif keyword_dict == news_obj:
        news_combo = []
        for words in tl.product(*t):
            news_combo.append(words)
        return news_combo

#turn tuple into list
def tuple_to_list(combos):
    if combos == gnews_combo:
        gnews_combo_list = []
        for combo in gnews_combo:
            list1 = []
            for keyword in combo:
                list1.append(keyword)
            gnews_combo_list.append(list1)
        return gnews_combo_list
    elif combos == news_combo:
        news_combo_list = []
        for combo in news_combo:
            list1 = []
            for keyword in combo:
                list1.append(keyword)
            news_combo_list.append(list1)
        return news_combo_list
    
#creating a function to make the combinations readable for the parameters
def query_able(combo_list):
    #creating dictionary to store values     
    i = 1
    if combo_list == gnews_combo_list:
        gnews_query = {}
    #converting list into string for parameter, q 
        for single_list in combo_list:
            param_string = " ".join(single_list).strip(',')
            gnews_query[i] = param_string
            i += 1
        return gnews_query
    elif combo_list == news_combo_list:
        news_query={}
        for single_list in combo_list:
            param_string = " ".join(single_list).strip(',')
            news_query[i] = param_string
            i += 1
        return news_query
    
#function for receiving information via api
def pull_news(query_dict):
    for api in apis:
        if api == 'news':
            news_list = []
            for combo in news_query:
                parameters = {
                'q': news_query[combo], 'apiKey':news_key,'country': 'us'
                }
                # Make the request
                response = requests.get(news_url, params=parameters)
                # Convert the response to JSON format and pretty print it
                news_response_json = response.json()
                pprint.pprint(news_response_json)
                news_list.append(news_response_json['articles'])
            return news_list
                
        elif api == 'gnews':
            gnews_list = []
            for combo in gnews_query:
                parameters = {
                    'q':gnews_query[combo], 'token':gnews_key,'country': 'us', 'language':'en'
                }
                # Make the request
                response = requests.get(gnews_url, params=parameters)
                # Convert the response to JSON format and pretty print it
                news_response_json = response.json()
                pprint.pprint(news_response_json)
                gnews_list.append(news_response_json['articles'])
            return gnews_list

### Define function for api calls

In [11]:
def api_call(file_path):
    
    #read the csv into the environment
    csv = pd.read_csv(file_path)
    #creating a dictionary key value structure
    global obj
    obj = {} 
    for i in range(len(csv.columns)):
        obj[str(i)] = []
    display(obj) 
    
    #populating the new structure
    for list in range(len(csv.columns)):
        for word in csv.iloc[:,list]:
            if str(word) != 'nan':
                obj[str(list)].append(word)
    display(obj)
    
    #adding special characters to words for News API parameter and gnews parameter
    global news_obj
    news_obj = {} 
    for i in range(len(csv.columns)):
        news_obj[str(i)] = []

    for api in apis: #iterate through both apis
        if api == 'news': #if we are using the news api
            for columns in obj: #each columns of related keyswords
                if columns == '0': #choosing the first column
                    new_list = []
                    for i in range(len(obj[columns])): #iterating through the values for each key
                        new_word = '+' + obj['0'][i] + ' AND'
                        new_list.append(new_word)
                    news_obj['0'] = new_list   
                elif columns == str(len(obj)-1):
                    for i in range(len(obj[columns])):
                        news_obj[columns] = obj[columns]
                    
                
        
                else:
                    new_list = []
                    for i in range(len(obj[columns])): #iterating through the values for each key
                        new_word = obj[columns][i] + ' AND'
                        new_list.append(new_word)
                    news_obj[columns] = new_list   
                    
                    
        elif api == 'gnews':
            continue
            
    print('This is for gnews')
    print(obj)
    print("\n")
    print('This is for news')
    print(news_obj)
    
    #calling combinations() function
    global gnews_combo
    global news_combo
    gnews_combo = combinations(obj)
    news_combo = combinations(news_obj)
    
    #calling tuples_to_list() function
    global gnews_combo_list
    global news_combo_list
    gnews_combo_list = tuple_to_list(gnews_combo)
    news_combo_list = tuple_to_list(news_combo)
    
    #calling query_able() function
    global gnews_query
    global news_query
    gnews_query = query_able(gnews_combo_list)
    news_query = query_able(news_combo_list)
    
    display(news_query) 
    
    #calling pull_news() function
    global gnews_list
    global news_list
    gnews_list = pull_news(gnews_query)
    news_list = pull_news(news_query)
    
    #pulling unique titles and then compiling titles with the corresponding info
    #turning news into a flat list

    news_flat_list = []
    for x in news_list:
        for y in x:
            news_flat_list.append(y)


    #turning gnews into a flat list
    gnews_flat_list = []
    for x in gnews_list:
        for y in x:
            gnews_flat_list.append(y)
        
        
    #putting title as the key for each value
    gnewsd = {}
    newsd = {}


    for x in gnews_flat_list:
        gnewsd[x['title']] = x

    for x in news_flat_list:
        newsd[x['title']] = x    
    
    
    #getting list of unique article names
    gnews_article_names = [] #GNEWS
    for x in gnews_flat_list:
        if x['title'] not in gnews_article_names:
            gnews_article_names.append(x['title'])


    #print(gnews_article_names)

    print('There are ' , len(gnews_article_names) , ' unique articles from gnews.')

    news_article_names = [] #NEWSAPI
    for x in news_flat_list:
        if x['title'] not in news_article_names:
            news_article_names.append(x['title'])

    #print(news_article_names)
    print('There are ' , len(news_article_names) , ' unique articles from news.')

    #getting combined list of article names
    combined_article_name = news_article_names + gnews_article_names
    print('There are ' , len(combined_article_name) , 'articles combined')

    #pulling unique article names into a separate list:
    unique_list = []
    for x in combined_article_name:
        if x not in unique_list:
            unique_list.append(x)
    print('There are ' , len(unique_list) , ' unique articles combined')

    #display(unique_list)
    #defining variables (included variables: title, description, source, url )



    #creating a dataframe
    data = pd.DataFrame()

    data['title'] = unique_list

    #adding description to the dataframe
    for x in unique_list:
        if x in gnewsd:
            description = gnewsd[x]['description']
            title_index = unique_list.index(x)
            data.loc[title_index,'description'] = gnewsd[x]['description']
        elif x in newsd:
            description = newsd[x]['description']
            title_index = unique_list.index(x)
            data.loc[title_index,'description'] = newsd[x]['description']
        
    #adding url to the dataframe
    for x in unique_list:
        if x in gnewsd:
            url = gnewsd[x]['url']
            title_index = unique_list.index(x)
            data.loc[title_index,'url'] = gnewsd[x]['url']
        elif x in newsd:
            url = newsd[x]['url']
            title_index = unique_list.index(x)
            data.loc[title_index,'url'] = newsd[x]['url']

    #adding url to the dataframe
    for x in unique_list:
        if x in gnewsd:
            url = gnewsd[x]['url']
            title_index = unique_list.index(x)
            data.loc[title_index,'url'] = gnewsd[x]['url']
        elif x in newsd:
            url = newsd[x]['url']
            title_index = unique_list.index(x)
            data.loc[title_index,'url'] = newsd[x]['url']
        
    #adding source {'name'} to the dataframe
    for x in unique_list:
        if x in gnewsd:
            source = gnewsd[x]['source']
            title_index = unique_list.index(x)
            data.loc[title_index,'source'] = gnewsd[x]['source']['name']
        elif x in newsd:
            source = newsd[x]['source']
            title_index = unique_list.index(x)
            data.loc[title_index,'source'] = newsd[x]['source']['name']        
        
        
    display(data)
        

    
    #turning dataframe into csv file
    data.to_csv('news_articlesV3.csv')

### Providing keywords HERE

In [13]:
api_call('Test2.csv') #input file path into the function


{'0': [], '1': [], '2': [], '3': []}

{'0': ['COVID-19'],
 '1': ['improvise', 'improvisation'],
 '2': ['hospital', 'clinic'],
 '3': ['US']}

This is for gnews
{'0': ['COVID-19'], '1': ['improvise', 'improvisation'], '2': ['hospital', 'clinic'], '3': ['US']}


This is for news
{'0': ['+COVID-19 AND'], '1': ['improvise AND', 'improvisation AND'], '2': ['hospital AND', 'clinic AND'], '3': ['US']}


{1: '+COVID-19 AND improvise AND hospital AND US',
 2: '+COVID-19 AND improvise AND clinic AND US',
 3: '+COVID-19 AND improvisation AND hospital AND US',
 4: '+COVID-19 AND improvisation AND clinic AND US'}

{'articleCount': 0, 'articles': [], 'timestamp': 1591336789}
{'articleCount': 1,
 'articles': [{'description': 'Source: Marcy Sanchez, Landstuhl Regional '
                              'Medical Center, U.S. Army Photo COVID-19 has '
                              'forced us to improvise ... member out of the '
                              'mental health clinic? The military culture '
                              'itself creates ...',
               'image': None,
               'publishedAt': '2020-05-13 13:28:00 UTC',
               'source': {'name': 'Psychology Today',
                          'url': 'https://www.psychologytoday.com'},
               'title': 'Is Telehealth an Answer to Military Mental Health '
                        'Stigma?',
               'url': 'https://www.psychologytoday.com/us/blog/alpha-blog-charlie/202005/is-telehealth-answer-military-mental-health-stigma'}],
 'timestamp': 1591336790}
{'articleCount': 2,
 'articles': [{'description': 'Jessica Coughlan,

,title,description,url,source
0,Is Telehealth an Answer to Military Mental Hea...,"Source: Marcy Sanchez, Landstuhl Regional Medi...",https://www.psychologytoday.com/us/blog/alpha-...,Psychology Today
1,Improvisation and medical mysteries in the COV...,"Jessica Coughlan, a cardiac nurse at Tufts Med...",https://www.msn.com/en-us/health/medical/impro...,MSN
